In [2]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent
from typing import List, Dict


In [7]:
from docx import Document
from docxcompose.composer import Composer

In [8]:
from docx import Document

doc = Document("/Users/avicenne/Documents/python/AnnotateWithAI/temp/incorrect_facts_document.docx")

for para in doc.paragraphs:
    print(para.text)

Sample English Document with Inaccuracies
The sun rises in the west every morning and sets in the east.
Water boils at 50 degrees Celsius under normal atmospheric pressure.
Albert Einstein was born in 1979 and invented the computer.
The capital of France is Madrid, which is known for its Eiffel Tower.
Shakespeare wrote the novel 'War and Peace' in the 19th century.
The Great Wall of China is visible from the Moon with the naked eye.
There are 500 states in the United States of America, each with its own president.
Penguins can fly thousands of miles every winter to reach the Arctic.
The human body contains 300 bones that never change throughout life.
Electric cars are powered entirely by gasoline and have no batteries.


In [9]:
def annotate_docx(paragraph_number: int, comments: str, filename: str) -> str:
    """
    add comment to a docx

    Args:
        paragraph_number (int): the paragraph number to put a comment
        comments (str): the recommandation to put
        filename (str): name of the file without extension
        
    Return:
        str
    """
    
    doc = Document(f"output/{filename}_commented.docx")
    p = doc.paragraphs[paragraph_number]
    
    composer = Composer(doc)
    composer.add_comment(p, "ChatGPT", comments)
    
    print("comment added")

    doc.save(f"output/{filename}_commented.docx") 

    print("document saved")
    
    return "comment added and document saved"


In [14]:
llm = ChatOllama(
    model= "deepseek-r1:1.5b",
    temperature=0.7,
)

annotate_agent = create_react_agent(
    model= llm,
    tools=[annotate_docx],
    prompt=(
        """
        You are a helpful agent how analyze a given document and provide recommandation on the paragraphs content
        
        Instructions:
            - adapt your analysis according to the user's request
            - once you have detected a modification to make, select paragraphs number associated to your recommandation
            - use your tool to put the comment on the file, one comment should be put one at a time
            
        Tool:
            - give paragraph number, the associated recommandation and the filename
        """
    ),
    name="annotate_agent",
    
)

In [15]:
content = ""
doc = Document("/Users/avicenne/Documents/python/AnnotateWithAI/temp/incorrect_facts_document.docx")

for i, para in enumerate(doc.paragraphs):
    content += f"[Paragraph number: {i}]: {para.text}"
    
filename = "incorrect_facts_document"

print(content)

query = f"""
Here is the content of document to correct: {content}
Please add comments to pinpoints all the incorrect fact in the doc
Here is the document file name: {filename}, don't mind for the extension, it is odcx but don't specify it when calling your tool

"""

annotate_agent.invoke({"role": "user", "content": query})

[Paragraph number: 0]: Sample English Document with Inaccuracies[Paragraph number: 1]: The sun rises in the west every morning and sets in the east.[Paragraph number: 2]: Water boils at 50 degrees Celsius under normal atmospheric pressure.[Paragraph number: 3]: Albert Einstein was born in 1979 and invented the computer.[Paragraph number: 4]: The capital of France is Madrid, which is known for its Eiffel Tower.[Paragraph number: 5]: Shakespeare wrote the novel 'War and Peace' in the 19th century.[Paragraph number: 6]: The Great Wall of China is visible from the Moon with the naked eye.[Paragraph number: 7]: There are 500 states in the United States of America, each with its own president.[Paragraph number: 8]: Penguins can fly thousands of miles every winter to reach the Arctic.[Paragraph number: 9]: The human body contains 300 bones that never change throughout life.[Paragraph number: 10]: Electric cars are powered entirely by gasoline and have no batteries.


ResponseError: registry.ollama.ai/library/deepseek-r1:1.5b does not support tools (status code: 400)